# <span style="color:blue"> Binary Segmentation with the U-Net Model

**Segmenting a single object** - the ```Multiple Sclerosis``` dataset

In this notebook, we will train a U-Net model to perform semantic segmentation on RGB images of cars from the **Multiple Sclerosis dataset**. The objective of this task is to accurately separate the lesions from their background.

<span style="color:red"> **Before proceeding**, ensure the dataset is available at ```D:\PIR_SEP\Datasets\BRAIN_MRI_SLICES```  

<span style="color:black">

The **U-Net** architecture is ideal for this task because it combines a contracting path (encoder) and an expanding path (decoder). This design effectively captures both high-level global features and fine-grained local details.

This notebook will guide you through the following steps:
>1. **Imports:** Setting up required libraries and packages.
>2. **Data Loading and Visualization:** Understanding the dataset and preprocessing it for the task.
>3. **Network Architecture Definition:** Implementing the U-Net model.
>4. **Model Training:** Optimizing the network with backpropagation.
>5. **Inference and Evaluation:** Testing the model on unseen data and assessing its performance using relevant metrics.

---

### <span style="color:brown"> **What is Segmentation?**

Segmentation involves assigning a class label to every pixel in an image.

- **Binary Segmentation:** Identifies a single object class (e.g., car) against the background.
- **Multi-Class Segmentation:** Differentiates between multiple object classes.

---

### <span style="color:brown"> **Semantic vs. Instance Segmentation**

When multiple objects of the same class appear in an image:
- **Semantic Segmentation:** Assigns the same label to all objects of the same class.
- **Instance Segmentation:** Differentiates and labels each object instance separately, enabling object counting.

Instance separation can sometimes be performed using traditional image processing techniques after semantic segmentation.

<img src="figures/SemanticIntanceSegmentation.png" alt="Semantic vs Instance Segmentation" style="width: 75%;"/>

**In this notebook, we focus on semantic segmentation** using the U-Net deep learning architecture.

---

### <span style="color:brown"> **A Quick Overview of U-Net**

The U-Net architecture is a fully **convolutional network** designed for segmentation tasks. Its key features include:
- **Skip connections:** Directly link the encoder and decoder layers to preserve spatial details and scale information.
- **Symmetrical design:** Combines downsampling for context (**encoder in orange**) with upsampling for precise localization (**decoder in green**).

<img src="figures/UNet.png" alt="U-Net Architecture" style="width: 70%;"/>


## <span style="color:brown"> 1. Imports 
Before we begin, we will import the necessary libraries for data handling, image transformations, and model training using PyTorch.

In [6]:
# Import libraries
# PyTorch for building and training neural networks
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

from torch import Tensor

# Torchvision for data transformations
import torchvision
from torchvision import datasets, transforms

# Torchinfo to display model summaries
from torchinfo import summary

# Numpy and Pathlib for data handling
import numpy as np
from pathlib import Path

# Tqdm for progress bars
from tqdm.auto import tqdm

# Import the 'os' module, which provides functions for interacting with the operating system.
# This module is useful for tasks like handling file paths, creating directories, and managing files.
import os

# Import specific types from the 'typing' module, which provides support for type hints in Python.
# 'Optional' is used for indicating that a variable might be of a certain type or None.
# 'Union' allows a variable to be one of several types.
# 'Tuple' is used to specify a fixed-size sequence of elements of specified types.
from typing import Optional, Union, Tuple

# Monai for medical imaging, specific to preprocessing and loss functions
import monai
from monai.transforms import LoadImage
# from monai.losses import DiceLoss

# Matplotlib for data visualization
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

# Import the 'glob' module, which provides a way to retrieve files matching a specified pattern. 
# This is useful for loading multiple image and mask files from directories.
import glob
# Import the 'Image' class from the 'PIL' (Python Imaging Library) module.
# This class allows us to open, manipulate, and save various image file formats.
from PIL import Image

# Check and display library versions for reproducibility
print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"Monai version: {monai.__version__}")

import copy
import os
import random
import shutil
import zipfile
from math import atan2, cos, sin, sqrt, pi, log

#import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
from numpy import linalg as LA
from torch import optim, nn
from torch.utils.data import DataLoader, random_split
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from tqdm import tqdm


PyTorch version: 2.6.0+cu118
Torchvision version: 0.21.0+cu118
Numpy version: 1.26.4
Monai version: 1.4.1rc1+46.gb58e883c


In [7]:

# Device setup
# Use GPU if available for faster training, otherwise fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(torch.cuda.is_available())
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
!which python



Using device: cuda
True
2.6.0+cu118
11.8
90100
/c/Users/sofia/anaconda3/envs/monai-dev/python


--- 
## <span style="color:brown"> 2. Load and Visualize the Data
In this section, we load and visualize RGB images from the carvana dataset.

Datasets often require preprocessing steps to prepare the data for training. Here, we will visualize a few examples to understand what our model will learn to segment. 
    

### <span style="color:brown"> 2.1 Create Dataset objects
`SepDataset`  will be used to load the data during training, validating and testing.

In [8]:
from torchvision import transforms
from PIL import Image
import torch
from torch.utils.data import Dataset

# Transformation personnalis√©e pour redimensionner l'image et le masque
class ResizeImageAndMask:
    def __init__(self, image_size, mask_size):
        #on redimensionne pour :
        #Uniformiser toutes les images (et masques) √† une taille standard
        #Permettre des batchs (traiter plusieurs images √† la fois)
        #Faire tourner le mod√®le sans crash
        
        #Quand tu redimensionnes une image, tu perds des pixels (ou tu en inventes, si tu agrandis).
        #‚Üí L‚Äôinterpolation est juste la m√©thode utilis√©e pour remplir les trous ou lisser la transition.

        # Redimensionner l'image avec interpolation bilin√©aire, fait une moyenne des pixels voisins
        self.image_resize = transforms.Resize(image_size, interpolation=Image.BILINEAR)
        # Redimensionner le masque avec interpolation nearest (voisinage), plus proche voisin, si tu fais une interpolation bilin√©aire, tu risques d‚Äôavoir des valeurs entre 0 et 1, genre 0.3, 0.8... ce qui n‚Äôa aucun sens pour un masque binaire
        self.mask_resize = transforms.Resize(mask_size, interpolation=Image.NEAREST)

    def __call__(self, sample):
        # Sample est un tuple (image, mask)
        image, mask = sample
        image = self.image_resize(image)
        mask = self.mask_resize(mask)
        return image, mask


class SepDataset(Dataset):
    #  def __init__(self, image_paths, mask_paths, resized_width, resized_height, limit=None):
    #limit = nombre limit√© de couples image/mask
    def __init__(self, root_path, resized_width, resized_height,limit=None):
        self.root_path = root_path
        self.limit = limit
        # Liste des chemins des images et des masques
        #trier les images et les masques (pour que image <-> mask reste align√©) et en prendre jusqu'√† la limite d√©finie
        self.images = sorted(
            [os.path.join(root_path, "train_data", i) 
             for i in os.listdir(os.path.join(root_path, "train_data")) 
             if i.endswith(".png")]
        )[:self.limit]
        
        self.masks = sorted(
            [os.path.join(root_path, "train_seg", i) 
             for i in os.listdir(os.path.join(root_path, "train_seg")) 
             if i.endswith(".png")]
        )[:self.limit]

        #d√©finir la taille des images et des masques
        self.resized_width = resized_width
        self.resized_height = resized_height
        # D√©finir la transformation compos√©e
        self.transform = transforms.Compose([
             ResizeImageAndMask((resized_width, resized_height), (resized_width, resized_height)),  # Redimensionner l'image et le masque
 
        ])

        if self.limit is None:
            self.limit = len(self.images)

    def __getitem__(self, index):
        # Charger l'image et le masque
        img = Image.open(self.images[index]).convert("L")
        mask = Image.open(self.masks[index]).convert("L") #luminance = noir et blanc

        # Appliquer la transformation compos√©e (l'image et le masque doivent √™tre pass√©s comme un tuple)
        img, mask = self.transform((img, mask))  # Passer le tuple √† la transformation

        # Convertir l'image en tenseur
        img = transforms.ToTensor()(img)

        # V√©rifier la valeur maximale de l'image apr√®s la conversion en tenseur
        if img.max() > 1.0:  # Si l'image est dans la plage [0, 255]
            img = img / 255.0  # Normaliser les pixels dans la plage [0, 1]

        # Normalisation de l'image entre [-1, 1]
        img = (img - 0.5) / 0.5  # Normalisation de l'image

        # Convertir le masque en tenseur et binariser
        mask = transforms.ToTensor()(mask)  # Convertir le masque en tenseur
        mask = (mask > 0).float()  # Binariser le masque (0 ou 1)

        # Retourner l'image transform√©e, le masque transform√© et le chemin de l'image
        return img, mask, self.images[index]

    def __len__(self):
        return min(len(self.images), self.limit)


### <span style="color:brown"> 2.2 - Load Files and Create Datasets

The ```SepDataset``` class is responsible for loading and formatting the data. It ensures that:
- The **size** of the images and masks (labels) is adjusted to the desired dimensions using resizing transformations.
- The **range** of pixel values is normalized:
  - **Images:** Normalized to the range [-1, 1] after being scaled to [0, 1].
  - **Masks:** Binarized to contain only 0 or 1 values after conversion to tensors.
- The **type** of images and masks is compatible with PyTorch:
  - Images are converted to `float32` tensors.
  - Masks are also converted to `float32` tensors but remain binary.

#### Using the Entire Dataset
To process the entire dataset, simply **omit the `limit` parameter** when calling the ```SepDataset``` class. If a value is provided for `limit`, only the specified number of samples will be loaded, which can be useful for debugging or testing.

```python
dataset = SepDataset(root_path="D:\PIR_SEP\Datasets\BRAIN_MRI_SLICES", resized_width=128, resized_height=128, limit=None)


In [9]:
from torch.utils.data import Dataset
from PIL import Image
import os
import torchvision.transforms as T

width=304   # width of images/masks processed by the network
height=304  # height of images/masks processed by the network
#print(os.listdir("./."))

root_dir = r"D:\PIR_SEP\Datasets\datagood2"

def dataset_axis(root_dir, axis, transform=None):
    """
    root_dir : dossier racine contenant 'axial', 'coronal', 'sagittal'
    axis : l'axe √† charger
    transform : transformations √† appliquer aux images et masques
    """
    
     # On trouve les images et les masques:
    train_dir = os.path.join(root_dir, axis)
    train_dataset = SepDataset(train_dir,resized_width=width,resized_height=height)        # Full database
    
    #Si tu n'utilisais pas de g√©n√©rateur, la division serait diff√©rente √† chaque ex√©cution du code.
    generator = torch.Generator().manual_seed(25)

    #diviser le dataset en dataset d'entra√Ænement et de test de mani√®re random avec 0.8 train et 0.2 test en utilisant les g√©n√©rateur
    train_dataset, test_dataset = random_split(train_dataset, [0.8, 0.2], generator=generator)
    print("Size dataset :",len(train_dataset)+len(test_dataset))
    print("Size train_dataset :",len(train_dataset))
    print("Size test_dataset :",len(test_dataset))
    
    #diviser le dataset de test en dataset de test et de validation √† 0.5 et 0.5 en utilisant le g√©n√©rateur
    test_dataset, val_dataset = random_split(test_dataset, [0.5, 0.5], generator=generator)
    print("Size train_dataset :",len(train_dataset))
    print("Size test_dataset :", len(test_dataset))
    print("Size val_dataset :", len(val_dataset))

    return train_dataset, val_dataset, test_dataset

# Stocker les donn√©es de chaque axe dans un dictionnaire
# On stocke les datasets dans un dictionnaire
all_datasets = {}
for axis in ["red", "green", "yellow"]:
    train_set, test_set, val_set = dataset_axis(root_dir, axis)
    all_datasets[axis] = {"train": train_set, "test": test_set, "val": val_set}

device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    num_workers = torch.cuda.device_count() * 4 - 1 #L'id√©e de multiplier par 4 et soustraire 1 est une m√©thode pour utiliser efficacement les ressources disponibles.
#num_workers = 1  # cmu
print("device :",device)
print("num_workers :",num_workers)

LEARNING_RATE = 3e-4 #Le learning rate (taux d'apprentissage) d√©termine √† quel point les poids du mod√®le doivent √™tre ajust√©s pendant l'entra√Ænement √† chaque it√©ration.
BATCH_SIZE = 8  # 1   #  4, 8, 16, 32 Le batch size (taille du lot) d√©termine combien d'exemples seront trait√©s simultan√©ment dans chaque passage du mod√®le pendant l'entra√Ænement.


Size dataset : 8645
Size train_dataset : 6916
Size test_dataset : 1729
Size train_dataset : 6916
Size test_dataset : 865
Size val_dataset : 864
Size dataset : 8645
Size train_dataset : 6916
Size test_dataset : 1729
Size train_dataset : 6916
Size test_dataset : 865
Size val_dataset : 864
Size dataset : 8645
Size train_dataset : 6916
Size test_dataset : 1729
Size train_dataset : 6916
Size test_dataset : 865
Size val_dataset : 864
device : cuda
num_workers : 3


In [10]:
# Charger les datasets pour un axe donn√© (ex. red)
train_dataset, val_dataset, test_dataset = dataset_axis(root_dir, "red")

#Cr√©ation des objets DataLoader pour charger les donn√©es d'entra√Ænement, de validation et de test dans des mini-lots (batches) pendant l'entra√Ænement de ton mod√®le.
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True) #Cela signifie que les donn√©es seront m√©lang√©es √† chaque √©poque pour √©viter que le mod√®le apprenne de mani√®re s√©quentielle et n'int√®gre pas des biais li√©s √† l'ordre des donn√©es.
val_dataloader = DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,shuffle= False) #Les donn√©es de validation ne sont pas m√©lang√©es. Cela permet de toujours tester le mod√®le sur les m√™mes donn√©es √† chaque √©poque pour une √©valuation stable.
test_dataloader = DataLoader(dataset=test_dataset,batch_size=BATCH_SIZE, shuffle=False) #Comme pour la validation, les donn√©es de test ne sont pas m√©lang√©es pour garantir une √©valuation coh√©rente.

Size dataset : 8645
Size train_dataset : 6916
Size test_dataset : 1729
Size train_dataset : 6916
Size test_dataset : 865
Size val_dataset : 864


In [11]:
# V√©rification du DataLoader
for img, mask, path in  test_dataloader:
    print(f'Image batch shape: {img.shape}') #nbre d'images dans le batch, nombre de canaux (3 pour RGB, 1 pour L), hauteur, largeur)
    print(f'Mask batch shape: {mask.shape}')
    print(f'Image path: {path[0]}')  # Afficher un chemin d'image pour v√©rifier
    break  # Juste pour v√©rifier une premi√®re it√©ration


Image batch shape: torch.Size([8, 1, 304, 304])
Mask batch shape: torch.Size([8, 1, 304, 304])
Image path: D:\PIR_SEP\Datasets\datagood2\red\train_data\Patient-19_19-Flair_red_124.png


In [24]:
for axis in ["red", "green", "yellow"]:
    print(f"\n Axe : {axis.upper()}")
    
    # R√©cup√©rer le dataloader de test correspondant
    test_dataset = all_datasets[axis]["test"]
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    # Cr√©er un it√©rateur
    test_dataloader_iter = iter(test_dataloader)
    
    # Prendre un batch
    batch_data = next(test_dataloader_iter)
    image, label = batch_data[0].to(device), batch_data[1].to(device)
    
    # Afficher infos
    print("image.shape :", image.shape)
    print("label.shape :", label.shape)
    print("torch.max(label) :", torch.max(label))
    print("torch.max(image) :", torch.max(image))



 Axe : RED
image.shape : torch.Size([1, 1, 304, 304])
label.shape : torch.Size([1, 1, 304, 304])
torch.max(label) : tensor(1., device='cuda:0')
torch.max(image) : tensor(0.9451, device='cuda:0')

 Axe : GREEN
image.shape : torch.Size([1, 1, 304, 304])
label.shape : torch.Size([1, 1, 304, 304])
torch.max(label) : tensor(1., device='cuda:0')
torch.max(image) : tensor(0.8510, device='cuda:0')

 Axe : YELLOW
image.shape : torch.Size([1, 1, 304, 304])
label.shape : torch.Size([1, 1, 304, 304])
torch.max(label) : tensor(1., device='cuda:0')
torch.max(image) : tensor(0.8510, device='cuda:0')


In [13]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# üîç Affichage
figure = plt.figure(figsize=(12, 9))

def display_image_and_label(image, label, index, title_prefix):
    image = image.detach().cpu().numpy()
    label = label.detach().cpu().numpy()
    image = np.squeeze(image)
    label = np.squeeze(label)
    
    ax_img = figure.add_subplot(3, 6, index * 2 + 1)
    ax_img.imshow(image, cmap="gray")
    ax_img.set_title(f"{title_prefix} Image")
    ax_img.axis("off")
    
    ax_mask = figure.add_subplot(3, 6, index * 2 + 2)
    ax_mask.imshow(label, cmap="jet", alpha=1)
    ax_mask.set_title(f"{title_prefix} Label")
    ax_mask.axis("off")

row_index = 0
for axis_name, splits in all_datasets.items():
    for split_name, dataset in splits.items():
        idx = np.random.randint(len(dataset))
        sample = dataset[idx]
        image = sample[0]
        label = sample[1]
        display_image_and_label(image, label, row_index, f"{axis_name.capitalize()} {split_name.capitalize()}")
        row_index += 1

plt.tight_layout()
plt.show()


---
## <span style="color:brown"> 3. Define the Network [Architecture](http://pytorch.org/docs/stable/nn.html)

The U-Net model for binary segmentation takes a batch of input images and processes each image to output **a corresponding probability map**. Each pixel in this map represents the likelihood that the pixel belongs to the target object class. 

Since we‚Äôre performing binary segmentation, the output for each image is a single-channel tensor, where each pixel value ranges between 0 and 1. A value closer to 1 indicates a higher probability that the pixel is part of the target object, while a value closer to 0 suggests background.

This approach allows the model to efficiently learn the spatial structure of the objects within the images, providing pixel-wise predictions for accurate segmentation.

### <span style="color:brown"> 3.1 Model Architecture Parameters

In [14]:
# Model Architecture Parameters
input_channels = 1    #e.g. 3 for RGB, 1 for grayscale input images
num_classes  = 1      #  e.g. 1 for binary segmentation (background vs object)
input_shape = (input_channels, width, height)  # This is the shape of the input image to the network
output_shape = (num_classes, width, height)  # This is the shape of the output mask
init_channels = 32              # This is the number of channels in the first layer of the network
# Nombre de filtres appliqu√©s √† l‚Äôimage dans la 1√®re couche
#Chaque filtre apprend √† d√©tecter quelque chose de diff√©rent :
#un peut d√©tecter des bords horizontaux,
#un autre les coins,
#un autre une texture sp√©cifique, etc.

### <span style="color:brown">3.2 Model Architecture</span>

In this section, two distinct model architectures are provided. However, you should only use one model for your task. Please select and activate the corresponding cell below based on your preferred model choice.

#### <span style="color:brown">Comparison of MONAI's `UNet` and Custom `UNet` Model in PyTorch</span>

When working with U-Net architectures for image segmentation, both **MONAI**'s `UNet` and a **custom PyTorch U-Net** can be used. However, they offer different levels of abstraction, flexibility, and usability. Below is a comparison of these two approaches:



##### <span style="color:brown">MONAI `UNet` Model</span> (from `monai.networks.nets`)

- **Purpose**: MONAI is a framework specifically designed for medical imaging tasks, built on top of PyTorch.
  
- **Ease of Use**: MONAI provides a high-level interface that simplifies the process of creating U-Net models. It allows users to quickly configure and instantiate a U-Net model by specifying only a few parameters, such as the number of input and output channels, the number of feature channels, and other configuration parameters like the stride and number of residual units.

- **Customizability**: While MONAI's `UNet` is highly configurable with parameters like `channels`, `strides`, and `num_res_units`, it hides some of the underlying complexity of U-Net, making it suitable for users who want an easy-to-implement model. 

- **Key Features**:
  - Optimized for 2D and 3D medical image segmentation.
  - Predefined configurations for common tasks, reducing setup time.
  - Focus on usability for medical AI tasks.


##### <span style="color:brown">Custom U-Net Model in PyTorch</span>

- **Purpose**: A custom U-Net implementation in PyTorch provides full flexibility, allowing users to define the architecture according to their specific needs. This model can be used for both general and specialized image segmentation tasks, not just medical imaging.

- **Flexibility**: The custom implementation allows for fine-grained control over every aspect of the network architecture. You can modify the number of layers, types of layers, normalization methods, upsampling techniques, and more.

- **Complexity**: Writing a custom U-Net requires more effort as you must manually define each block (e.g., convolution layers, downsampling, and upsampling). This gives more control over the implementation but increases the amount of code and complexity.

- **Customizability**: You can adjust various hyperparameters, such as the type of convolution (`Conv2d` or `ConvTranspose2d`), batch normalization, dropout rates, and even the use of bilinear interpolation or transposed convolutions for upsampling.

- **Key Features**:
  - Full control over architecture.
  - Greater flexibility to modify layer types and network structure.
  - Ideal for users who need customization for specific segmentation tasks.



##### <span style="color:brown">Comparison Table</span>

| **Feature**                        | **MONAI `UNet`**                                  | **Custom PyTorch `UNet`**                      |
|------------------------------------|--------------------------------------------------|-----------------------------------------------|
| **Ease of Use**                    | High (dedicated for medical image segmentation)   | Low (manual implementation required)          |
| **Flexibility**                    | Moderate (configurable parameters)               | High (full control over architecture)         |
| **Customizability**                | Low (limited to predefined blocks and parameters) | High (customize every part of the network)     |
| **Code Complexity**                | Low (simple instantiation)                       | High (requires manual architecture design)    |
| **Intended Use Case**              | Medical imaging segmentation                     | General segmentation tasks                    |


### <span style="color:brown"> Model Architecture: Custom Unet 

In [15]:
class UNet(nn.Module):
    """Architecture based on U-Net: Convolutional Networks for Biomedical Image Segmentation.

    References:
    - Paper that introduced the U-Net model: https://arxiv.org/abs/1505.04597
    """

    def __init__(
        self,
        input_shape: Tuple[int, ...],
        output_shape: Tuple[int, ...],
        init_channels: int = 32,
        use_batchnorm: bool = True,
        bilinear: bool = False,
        dropout: float = 0.0,
    ):
        """Initializes class instance.

        Args:
            input_shape: (in_channels, H, W), Shape of the input images.
            output_shape: (num_classes, H, W), Shape of the output segmentation map.
            init_channels: Number of output feature maps from the first layer, used to compute the number of feature
                maps in following layers.
            use_batchnorm: Whether to use batch normalization between the convolution and activation layers in the
                convolutional blocks.
            bilinear: Whether to use bilinear interpolation or transposed convolutions for upsampling.
            dropout: Probability of an element to be zeroed (e.g. 0 means no dropout).
        """
        super().__init__()
        in_channels = input_shape[0]
        out_channels = output_shape[0]

        self.layer1 = _DoubleConv(in_channels, init_channels // 2, dropout / 2, use_batchnorm)
        self.layer2 = _Down(init_channels // 2, init_channels, dropout, use_batchnorm)
        self.layer3 = _Down(init_channels, init_channels * 2, dropout, use_batchnorm)
        self.layer4 = _Down(init_channels * 2, init_channels * 4, dropout, use_batchnorm)

        self.layer5 = _Up(
            init_channels * 4, init_channels * 2, dropout, use_batchnorm, bilinear=bilinear
        )
        self.layer6 = _Up(
            init_channels * 2, init_channels, dropout, use_batchnorm, bilinear=bilinear
        )
        self.layer7 = _Up(init_channels, init_channels // 2, 0, use_batchnorm, bilinear=bilinear)

        self.layer9 = nn.Conv2d(init_channels // 2, out_channels, kernel_size=1)

        # Use Xavier initialisation for weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
                nn.init.xavier_uniform_(m.weight)

    def forward(self, x: Tensor) -> Tensor:
        """Defines the computation performed at every call.

        Args:
            x: (N, ``in_channels``, H, W), Input image to segment.

        Returns:
            (N, ``out_channels``, H, W), Raw, unnormalized scores for each class in the input's segmentation.
        """
        x1 = self.layer1(x)
        x2 = self.layer2(x1)
        x3 = self.layer3(x2)
        x4 = self.layer4(x3)

        out = self.layer5(x4, x3)
        out = self.layer6(out, x2)
        out = self.layer7(out, x1)

        return self.layer9(out)


class _DoubleConv(nn.Module):
    """Double Convolution and BN and ReLU.

    (3x3 conv -> BN -> ReLU) ** 2
    """

    def __init__(self, in_ch: int, out_ch: int, dropout_prob: float, use_batchnorm: bool):
        super().__init__()
        if use_batchnorm:
            self.net = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True),
                nn.Dropout(p=dropout_prob),
                nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True),
                nn.Dropout(p=dropout_prob),
            )
        else:
            self.net = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Dropout(p=dropout_prob),
                nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Dropout(p=dropout_prob),
            )

    def forward(self, x: Tensor) -> Tensor:
        return self.net(x)


class _Down(nn.Module):
    """Combination of MaxPool2d and DoubleConv in series."""

    def __init__(self, in_ch: int, out_ch: int, dropout_prob: float, use_batchnorm: bool):
        super().__init__()
        self.net = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            _DoubleConv(in_ch, out_ch, dropout_prob, use_batchnorm),
        )

    def forward(self, x: Tensor) -> Tensor:  
        return self.net(x)


class _Up(nn.Module):
    """Upsampling (by either bilinear interpolation or transpose convolutions).

    followed by concatenation of feature map from contracting path, followed by double 3x3
    convolution.
    """

    def __init__(
        self, in_ch, out_ch: int, dropout_prob: float, use_batchnorm: bool, bilinear: bool = False
    ):
        super().__init__()
        self.upsample = None
        if bilinear:
            self.upsample = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        else:
            self.upsample = nn.ConvTranspose2d(in_ch, in_ch // 2, kernel_size=2, stride=2)

        self.conv = _DoubleConv(in_ch, out_ch, dropout_prob, use_batchnorm)

    def forward(self, x: Tensor, connected_encoder_features: Tensor) -> Tensor:
        x = self.upsample(x)

        # Pad ``x`` to the size of ``connected_encoder_features``
        diff_h = connected_encoder_features.shape[2] - x.shape[2]
        diff_w = connected_encoder_features.shape[3] - x.shape[3]

        x = F.pad(x, [diff_w // 2, diff_w - diff_w // 2, diff_h // 2, diff_h - diff_h // 2])

        # Concatenate along the channels axis
        x = torch.cat([connected_encoder_features, x], dim=1)

        return self.conv(x)


# initialize the NN
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(input_shape=input_shape, output_shape=output_shape, init_channels=init_channels).to(device)

### <span style="color:brown"> Function: ```Summary``` </span>

The **summary** function provides a detailed overview of the model. Pay close attention to the following key information:

- **Number of parameters**: This indicates the total number of learnable parameters in the model, which can give you insight into its complexity.
- **Total model size**: This refers to the overall size of the model in memory, which is important for assessing its computational requirements and suitability for deployment.

Understanding these details can help you evaluate the efficiency and scalability of the model.

In [16]:
# Print the summary of the network
#summary_kwargs:	Un dictionnaire de param√®tres pour summary()
summary_kwargs = dict(
    col_names=["input_size", "output_size", "kernel_size", "num_params"], depth=5, verbose=0
)
#verbose=0	N‚Äôaffiche pas les entr√©es/sorties des modules tr√®s petits ou inutiles (plus clean)
#summary(model, input_size = (32,3, 64, 64),  device=device, **summary_kwargs)
summary(model, input_size = (BATCH_SIZE,1, width, height),  device=device, **summary_kwargs)
#**summary_kwargs: Tu le "d√©plies" pour passer chaque cl√©/valeur comme un argument √† la fonction

Layer (type:depth-idx)                        Input Shape               Output Shape              Kernel Shape              Param #
UNet                                          [8, 1, 304, 304]          [8, 1, 304, 304]          --                        --
‚îú‚îÄ_DoubleConv: 1-1                            [8, 1, 304, 304]          [8, 16, 304, 304]         --                        --
‚îÇ    ‚îî‚îÄSequential: 2-1                        [8, 1, 304, 304]          [8, 16, 304, 304]         --                        --
‚îÇ    ‚îÇ    ‚îî‚îÄConv2d: 3-1                       [8, 1, 304, 304]          [8, 16, 304, 304]         [3, 3]                    160
‚îÇ    ‚îÇ    ‚îî‚îÄBatchNorm2d: 3-2                  [8, 16, 304, 304]         [8, 16, 304, 304]         --                        32
‚îÇ    ‚îÇ    ‚îî‚îÄReLU: 3-3                         [8, 16, 304, 304]         [8, 16, 304, 304]         --                        --
‚îÇ    ‚îÇ    ‚îî‚îÄDropout: 3-4                      [8, 16, 304, 304]

In [17]:
# Print the summary of the network
summary_kwargs = dict(
    col_names=["input_size", "output_size", "kernel_size", "num_params"], depth=2, verbose=0
)
#summary(model, input_size = (32,3, 64, 64),  device=device, **summary_kwargs)
summary(model, input_size = (BATCH_SIZE,1, width, height),  device=device, **summary_kwargs)   

Layer (type:depth-idx)                        Input Shape               Output Shape              Kernel Shape              Param #
UNet                                          [8, 1, 304, 304]          [8, 1, 304, 304]          --                        --
‚îú‚îÄ_DoubleConv: 1-1                            [8, 1, 304, 304]          [8, 16, 304, 304]         --                        --
‚îÇ    ‚îî‚îÄSequential: 2-1                        [8, 1, 304, 304]          [8, 16, 304, 304]         --                        2,544
‚îú‚îÄ_Down: 1-2                                  [8, 16, 304, 304]         [8, 32, 152, 152]         --                        --
‚îÇ    ‚îî‚îÄSequential: 2-2                        [8, 16, 304, 304]         [8, 32, 152, 152]         --                        14,016
‚îú‚îÄ_Down: 1-3                                  [8, 32, 152, 152]         [8, 64, 76, 76]           --                        --
‚îÇ    ‚îî‚îÄSequential: 2-3                        [8, 32, 152, 152]      


## 4. Specify [Loss Function](http://pytorch.org/docs/stable/nn.html#loss-functions) and [Optimizer](http://pytorch.org/docs/stable/optim.html)

It's recommended that you use `BCEWithLogitsLoss`. 

`BCEWithLogitsLoss` combines **sigmoid activation** and **Binary Cross-Entropy (BCE) loss** into a single function. It is used when the **model's outputs** are raw logits (values before applying a sigmoid activation to convert them into probabilities).

`BCELoss` stands for **Binary Cross-Entropy Loss**. It is used when the model's output is already in the form of **probabilities** (values between 0 and 1).


- **`BCEWithLogitsLoss`**:
  - Accepts **logits** (raw model outputs).
  - Automatically applies the sigmoid function internally.
  - More **numerically stable** because sigmoid and BCE are combined into one operation.

- **`BCELoss`**:
  - Accepts **probabilities** (values between 0 and 1).
  - Requires **manual application of the sigmoid function** in the model‚Äôs output.


In [18]:
## Specify loss and optimization functions

# Parameters
learning_rate = 0.001
n_epochs = 1 # 5 # 20

# specify loss function
#criterion = DiceLoss()

criterion = nn.BCEWithLogitsLoss()
# specify optimizer
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
#optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

---
## <span style="color:brown"> 4. Train the Network

The steps for training/learning from a batch of data are described in the comments below:
1. Clear the gradients of all optimized variables
2. Forward pass: compute predicted outputs by passing inputs to the model
3. Calculate the loss
4. Backward pass: compute gradient of the loss with respect to model parameters
5. Perform a single optimization step (parameter update)
6. Update average training loss

The following loop trains for 3 epochs; feel free to change this number. For now, we suggest somewhere between 20-50 epochs. As you train, take a look at how the values for the training loss decrease over time. We want it to decrease while also avoiding overfitting the training data. 

### <span style="color:brown"> 4.1 Training loop

In [15]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

criterion = nn.BCEWithLogitsLoss()  # ta loss
axes = ['red', 'green', 'yellow']

train_losses_by_axis = {}
valid_losses_by_axis = {}

for axis in axes:
    print(f"\n Entra√Ænement pour l'axe : {axis.upper()}")

    # === Chargement des datasets ===
    train_dataset = all_datasets[axis]['train']
    val_dataset = all_datasets[axis]['val']

    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    # === Initialisation du mod√®le et de l'optimizer ===
    model = UNet(input_shape=input_shape, output_shape=output_shape, init_channels=init_channels).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

    # Suivi des pertes
    train_losses = []
    valid_losses = []
    best_valid_loss = float('inf')
    best_model = None
    best_epoch = 0

    for epoch in tqdm(range(n_epochs), desc=f"Training {axis.upper()}"):
        model.train()
        train_loss = 0.0

        for images, labels, _ in train_dataloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)

        avg_train_loss = train_loss / len(train_dataloader.dataset)
        train_losses.append(avg_train_loss)

        # === Validation ===
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for val_images, val_labels, _ in val_dataloader:
                val_images, val_labels = val_images.to(device), val_labels.to(device)
                val_outputs = model(val_images)
                loss = criterion(val_outputs, val_labels)
                val_loss += loss.item() * val_images.size(0)

        avg_val_loss = val_loss / len(val_dataloader.dataset)
        valid_losses.append(avg_val_loss)

        print(f"[{axis.upper()}][Epoch {epoch+1}/{n_epochs}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

        # Enregistrement du meilleur mod√®le
        if avg_val_loss < best_valid_loss:
            best_valid_loss = avg_val_loss
            best_model = model.state_dict()
            best_epoch = epoch + 1
            best_train_loss = avg_train_loss

    # === Sauvegarde du meilleur mod√®le ===
    model.load_state_dict(best_model)
    torch.save(best_model, f"best_unet_{axis}.pth")

    print(f" Meilleur mod√®le {axis.upper()} sauvegard√© ‚Äî Epoch {best_epoch}, Train Loss: {best_train_loss:.4f}, Val Loss: {best_valid_loss:.4f}")

    train_losses_by_axis[axis] = train_losses
    valid_losses_by_axis[axis] = valid_losses



 Entra√Ænement pour l'axe : RED


Training RED: 100%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 1/1 [11:04<00:00, 664.68s/it]


[RED][Epoch 1/1] Train Loss: 0.1245 | Val Loss: 0.0438
 Meilleur mod√®le RED sauvegard√© ‚Äî Epoch 1, Train Loss: 0.1245, Val Loss: 0.0438

 Entra√Ænement pour l'axe : GREEN


Training GREEN: 100%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 1/1 [07:00<00:00, 420.22s/it]


[GREEN][Epoch 1/1] Train Loss: 0.1994 | Val Loss: 0.0831
 Meilleur mod√®le GREEN sauvegard√© ‚Äî Epoch 1, Train Loss: 0.1994, Val Loss: 0.0831

 Entra√Ænement pour l'axe : YELLOW


Training YELLOW: 100%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 1/1 [10:27<00:00, 627.32s/it]

[YELLOW][Epoch 1/1] Train Loss: 0.1768 | Val Loss: 0.0666
 Meilleur mod√®le YELLOW sauvegard√© ‚Äî Epoch 1, Train Loss: 0.1768, Val Loss: 0.0666


### <span style="color:brown"> 4.2 Display train / valid curves

In [16]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 5))

for i, axis in enumerate(['red', 'green', 'yellow']):
    plt.subplot(1, 3, i + 1)
    
    train_loss = train_losses_by_axis[axis]
    val_loss = valid_losses_by_axis[axis]

    plt.plot(train_loss, label='Train Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.title(f"Loss - {axis.upper()}")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.suptitle("Courbes d'entra√Ænement et de validation pour chaque axe", fontsize=16, y=1.05)
plt.show()

---
## <span style="color:brown"> 5. Perform the inference on the test dataset


### <span style="color:brown"> 5.1 Load the weight of the best model
You could use other weights for your model...

In [19]:
from collections import defaultdict

# === Param√®tres du mod√®le ===
input_shape = (1, width, height)
num_classes = 1
output_shape = (num_classes, width, height)
init_channels = 32

# === Stockage des mod√®les et des pr√©dictions ===
models_by_axis = {}
predictions_by_slice = defaultdict(lambda: {'red': None, 'green': None, 'yellow': None, 'gt': None})

for axis in ['red', 'green', 'yellow']:
    print(f"üîÅ Chargement du mod√®le pour l‚Äôaxe : {axis.upper()}")

    model = UNet(input_shape=input_shape, output_shape=output_shape, init_channels=init_channels)
    weights_path = f"best_unet_{axis}.pth"
    state_dict = torch.load(weights_path, map_location=device)
    model.load_state_dict(state_dict)
    model.eval()
    models_by_axis[axis] = model


üîÅ Chargement du mod√®le pour l‚Äôaxe : RED
üîÅ Chargement du mod√®le pour l‚Äôaxe : GREEN
üîÅ Chargement du mod√®le pour l‚Äôaxe : YELLOW


Then, we perform the inference on a sample from the test dataset and plot the prediction.
Run the following cell many times to test several images.
**Take care of the dimensions of tensors and data !...**


### <span style="color:brown"> 5.2 Test and visualized the results on one sample of the test_dataset

We compare the computation of Dice and IoU metrics using two approaches:
1. **Custom Functions**: Defined directly in the code.
2. **MONAI Metrics**: Utilizing the built-in metrics provided by the MONAI library.

In [23]:
from collections import defaultdict

print("üìä Lancement de l‚Äô√©valuation uniquement pour le Patient 16")

import numpy as np

thresholds = np.arange(0.2, 0.51, 0.05)
fusion_methods = ["max", "mean", "vote", "min"]

for method in fusion_methods:
    print(f"\nüîß M√©thode : {method.upper()}")

    for threshold in thresholds:
        print(f"‚û°Ô∏è Seuil : {threshold:.2f}")

        patient_scores = defaultdict(lambda: {"dice": [], "iou": [], "hd": []})

        for i, (data_r, data_g, data_y) in enumerate(zip(
            test_dataloaders["red"], test_dataloaders["green"], test_dataloaders["yellow"]
        )):
            img_r, _, filename_r = data_r
            img_g, _, _ = data_g
            img_y, _, _ = data_y

            img_r = img_r.to(device)
            img_g = img_g.to(device)
            img_y = img_y.to(device)

            filename = Path(filename_r[0]).name
            mask_filename = infer_mask_filename(filename)
            patient_id = filename.split("-LesionSeg")[0]  # Ex: 'Patient-16_1'

            if not patient_id.startswith("Patient-16"):
                continue  # Ne traiter que Patient-16

            mask_path = mask_root_paths["red"] / mask_filename
            if not mask_path.exists():
                print(f"‚ùå Masque non trouv√© : {mask_path}")
                continue

            label = load_mask_as_tensor(mask_path).unsqueeze(0).unsqueeze(0).to(device)

            with torch.no_grad():
                out_r = torch.sigmoid(models_by_axis["red"](img_r))
                out_g = torch.sigmoid(models_by_axis["green"](img_g))
                out_y = torch.sigmoid(models_by_axis["yellow"](img_y))

            # === Fusion
            if method == "max":
                fused_pred = torch.max(torch.max(out_r, out_g), out_y)
            elif method == "mean":
                fused_pred = (out_r + out_g + out_y) / 3.0
            elif method == "min":
                fused_pred = torch.min(torch.min(out_r, out_g), out_y)
            elif method == "vote":
                fused_pred = majority_vote([out_r, out_g, out_y], threshold)
            else:
                raise ValueError(f"Fusion inconnue : {method}")

            if method != "vote":
                fused_bin = (fused_pred > threshold).float()
            else:
                fused_bin = fused_pred

            pred_np = fused_bin.cpu().numpy().squeeze()
            label_np = label.cpu().numpy().squeeze()

            d = dice_score(pred_np, label_np)
            i = iou_score(pred_np, label_np)
            h = compute_hd95(pred_np, label_np)

            patient_scores[patient_id]["dice"].append(d)
            patient_scores[patient_id]["iou"].append(i)
            if not np.isnan(h):
                patient_scores[patient_id]["hd"].append(h)

        # Moyennes uniquement pour Patient-16
        if not patient_scores:
            print("‚ö†Ô∏è Aucun masque trouv√© pour Patient-16 √† ce seuil.")
            continue

        dice_list = []
        iou_list = []
        hd_list = []

        for patient, scores in sorted(patient_scores.items()):
            dice_mean = np.mean(scores["dice"]) if scores["dice"] else np.nan
            iou_mean = np.mean(scores["iou"]) if scores["iou"] else np.nan
            hd_mean = np.mean(scores["hd"]) if scores["hd"] else np.nan

            dice_list.append(dice_mean)
            iou_list.append(iou_mean)
            if not np.isnan(hd_mean):
                hd_list.append(hd_mean)

        print(f"\nüß™ Moyennes Patient-16 - M√©thode : {method.upper()}, Seuil = {threshold:.2f}")
        print(f"  ‚û§ Dice moyen : {np.nanmean(dice_list):.4f}")
        print(f"  ‚û§ IoU  moyen : {np.nanmean(iou_list):.4f}")
        print(f"  ‚û§ HD95 moyen : {np.nanmean(hd_list):.2f}")


üìä Lancement de l‚Äô√©valuation uniquement pour le Patient 16

üîß M√©thode : MAX
‚û°Ô∏è Seuil : 0.20


KeyError: 'red'

---
### <span style="color:brown"> 5.3 Computing Metrics on the Entire Test Dataset

In this section, we evaluate the segmentation accuracy across the entire test dataset.

Various metrics can be used for this purpose (refer to your lecture notes for details). You are encouraged to explore and implement additional metrics to test their performance.

### Metrics Used:
- **Dice Score**: Used here to measure segmentation accuracy.
- **IoU Metric**: Add your own user-defined implementation to compute IoU.
- **MONAI Metrics**: Experiment with MONAI's built-in metrics for comparison.
    (https://docs.monai.io/en/stable/metrics.html

In [1]:
# Custom functions
# Implementation 2: Dice Coefficient ( PyTorch version- Tensor operations)
def dice_coefficient(prediction, target, epsilon=1e-07):
    prediction_copy = prediction.clone()
    prediction_copy[prediction_copy < 0] = 0
    prediction_copy[prediction_copy > 0] = 1
    intersection = abs(torch.sum(prediction_copy * target))
    union = abs(torch.sum(prediction_copy) + torch.sum(target))
    dice = (2. * intersection + epsilon) / (union + epsilon)
    return dice

# Implementation 1: Dice Score (NumPy Version - Array operations)
def dice_score(pred, label):
    smooth = 1e-6
    intersection = np.sum(pred * label)
    return 2. * intersection / (np.sum(pred) + np.sum(label) + smooth)
    
#  IoU Score (Intersection over Union)  (NumPy Version - Array operations)
def iou_score(pred, label):
    return jaccard_score(label.flatten(), pred.flatten())

test_running_loss = 0
test_running_dc1 = 0
test_running_dc2 = 0  # comparison with dice_score function

model.eval()
with torch.no_grad():
    for idx, img_mask in enumerate(tqdm(test_dataloader, position=0, leave=True)):
        img = img_mask[0].float().to(device)
        mask = img_mask[1].float().to(device)
        if idx==0:
            print("0. img.shape:", img.shape,"0. mask.shape:",mask.shape)
        y_pred = model(img)
        loss = criterion(y_pred, mask)
        dc1 = dice_coefficient(y_pred, mask)
        
        y_pred = (torch.sigmoid(y_pred) > 0.3).float()
        np_mask = mask.squeeze(dim=0).squeeze(dim=0).detach().cpu().numpy()  # Same here
        np_y_pred = y_pred.squeeze(dim=0).squeeze(dim=0).detach().cpu().numpy()  # Sequentially remove the first two dimensions
        if idx==0:
            print("1.y_pred.shape:", y_pred.shape, "1. mask.shape:", mask.shape,"1. np_y_pred:", pred.shape)
        dc2 = dice_score(np_y_pred, np_mask)
       
        test_running_loss += loss.item()
        test_running_dc1 += dc1.item()
        test_running_dc2 += dc2.item()

    test_loss = test_running_loss / (idx + 1)
    test_dc1 = test_running_dc1 / (idx + 1)
    test_dc2 = test_running_dc2 / (idx + 1)

print(f"Final Dice Metric 1 : {test_dc1:.3f}")
print(f"Final Dice Metric 2  : {test_dc2:.3f}")
print(f"test Loss : {test_loss:.3f}")

""" Essai Dice et IoU de Monai √† revoir, √† ne pas effacer !!!!"""
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from monai.networks.nets import UNet
from sklearn.metrics import jaccard_score
from monai.metrics import  DiceMetric, MeanIoU

# Check the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Metrics initialisation
dice_metric = DiceMetric(include_background=True, reduction="mean")
iou_metric = MeanIoU(include_background=True)  # No num_classes argument

# Dice and IoU metrics from monai, reset before use
dice_metric.reset()
iou_metric.reset()




NameError: name 'model' is not defined

### 5.4 Hidden

In [ ]:
from torch.utils.data import DataLoader
import os

# === Cr√©ation des dataloaders et collecte des pr√©dictions ===
num_workers = 0
print("num_workers =", num_workers)

for axis in ['red', 'green', 'yellow']:
    print(f"üì• Inf√©rence pour l‚Äôaxe {axis.upper()}")

    test_dataset = all_datasets[axis]['test']
    test_loader = DataLoader(
        test_dataset,
        batch_size=1,
        shuffle=False,
        pin_memory=torch.cuda.is_available(),
        num_workers=num_workers
    )

    model = models_by_axis[axis]

    for i, batch in enumerate(test_loader):
        img, mask = batch
        img = img.float().to(device)
        mask = mask.squeeze().cpu().numpy()
        with torch.no_grad():
            pred = model(img)
            pred = torch.sigmoid(pred).squeeze().cpu().numpy()

        # üîë Identifiant de slice (par exemple le nom de fichier sans extension)
        slice_id = test_dataset.samples[i][1].stem  # √Ä adapter si n√©cessaire
        predictions_by_slice[slice_id][axis] = pred
        predictions_by_slice[slice_id]['gt'] = mask


üîé RED ‚Äì √âvaluation pour seuil 0.20



üîé RED ‚Äì √âvaluation pour seuil 0.25



üîé RED ‚Äì √âvaluation pour seuil 0.30



üîé RED ‚Äì √âvaluation pour seuil 0.35



üîé RED ‚Äì √âvaluation pour seuil 0.40



üîé RED ‚Äì √âvaluation pour seuil 0.45



üîé RED ‚Äì √âvaluation pour seuil 0.50



üîé GREEN ‚Äì √âvaluation pour seuil 0.20



üîé GREEN ‚Äì √âvaluation pour seuil 0.25


GREEN - seuil 0.25:   7%|‚ñà‚ñà‚ñà‚ñà‚ñè                                                        | 60/864 [00:04<01:00, 13.20it/s]

In [2]:
from torch.utils.data import DataLoader
import os

# Configuration
num_workers=0
print("num_workers =", num_workers)

# Cr√©ation des test_dataloader par axe (batch=1 pour affichage)
test_dataloaders = {}
test_iterators = {}

for axis in ['red', 'green', 'yellow']:
    print(f"Cr√©ation du test_dataloader pour l'axe {axis.upper()}")
    
    test_dataset = all_datasets[axis]['test']
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=1,
        pin_memory=torch.cuda.is_available(),
        shuffle=False,
        num_workers=num_workers
    )
    
    # Stocker le dataloader et son it√©rateur
    test_dataloaders[axis] = test_loader
    test_iterators[axis] = iter(test_loader)


num_workers = 0
Cr√©ation du test_dataloader pour l'axe RED


NameError: name 'all_datasets' is not defined

In [25]:
from collections import defaultdict

print("üìä Lancement de l‚Äô√©valuation uniquement pour le Patient 16")

for method in fusion_methods:
    print(f"\nüîß M√©thode : {method.upper()}")

    for threshold in thresholds:
        print(f"‚û°Ô∏è Seuil : {threshold:.2f}")

        patient_scores = defaultdict(lambda: {"dice": [], "iou": [], "hd": []})

        for i, (data_r, data_g, data_y) in enumerate(zip(
            test_dataloaders["red"], test_dataloaders["green"], test_dataloaders["yellow"]
        )):
            img_r, _, filename_r = data_r
            img_g, _, _ = data_g
            img_y, _, _ = data_y

            img_r = img_r.to(device)
            img_g = img_g.to(device)
            img_y = img_y.to(device)

            filename = Path(filename_r[0]).name
            mask_filename = infer_mask_filename(filename)
            patient_id = filename.split("-LesionSeg")[0]  # Ex: 'Patient-16_1'

            if not patient_id.startswith("Patient-16"):
                continue  # Ne traiter que Patient-16

            mask_path = mask_root_paths["red"] / mask_filename
            if not mask_path.exists():
                print(f"‚ùå Masque non trouv√© : {mask_path}")
                continue

            label = load_mask_as_tensor(mask_path).unsqueeze(0).unsqueeze(0).to(device)

            with torch.no_grad():
                out_r = torch.sigmoid(models_by_axis["red"](img_r))
                out_g = torch.sigmoid(models_by_axis["green"](img_g))
                out_y = torch.sigmoid(models_by_axis["yellow"](img_y))

            # === Fusion
            if method == "max":
                fused_pred = torch.max(torch.max(out_r, out_g), out_y)
            elif method == "mean":
                fused_pred = (out_r + out_g + out_y) / 3.0
            elif method == "min":
                fused_pred = torch.min(torch.min(out_r, out_g), out_y)
            elif method == "vote":
                fused_pred = majority_vote([out_r, out_g, out_y], threshold)
            else:
                raise ValueError(f"Fusion inconnue : {method}")

            if method != "vote":
                fused_bin = (fused_pred > threshold).float()
            else:
                fused_bin = fused_pred

            pred_np = fused_bin.cpu().numpy().squeeze()
            label_np = label.cpu().numpy().squeeze()

            d = dice_score(pred_np, label_np)
            i = iou_score(pred_np, label_np)
            h = compute_hd95(pred_np, label_np)

            patient_scores[patient_id]["dice"].append(d)
            patient_scores[patient_id]["iou"].append(i)
            if not np.isnan(h):
                patient_scores[patient_id]["hd"].append(h)

        # Moyennes uniquement pour Patient-16
        if not patient_scores:
            print("‚ö†Ô∏è Aucun masque trouv√© pour Patient-16 √† ce seuil.")
            continue

        dice_list = []
        iou_list = []
        hd_list = []

        for patient, scores in sorted(patient_scores.items()):
            dice_mean = np.mean(scores["dice"]) if scores["dice"] else np.nan
            iou_mean = np.mean(scores["iou"]) if scores["iou"] else np.nan
            hd_mean = np.mean(scores["hd"]) if scores["hd"] else np.nan

            dice_list.append(dice_mean)
            iou_list.append(iou_mean)
            if not np.isnan(hd_mean):
                hd_list.append(hd_mean)

        print(f"\nüß™ Moyennes Patient-16 - M√©thode : {method.upper()}, Seuil = {threshold:.2f}")
        print(f"  ‚û§ Dice moyen : {np.nanmean(dice_list):.4f}")
        print(f"  ‚û§ IoU  moyen : {np.nanmean(iou_list):.4f}")
        print(f"  ‚û§ HD95 moyen : {np.nanmean(hd_list):.2f}")


üìä Lancement de l‚Äô√©valuation uniquement pour le Patient 16

üîß M√©thode : MAX
‚û°Ô∏è Seuil : 0.20


KeyError: 'red'

In [ ]:
# Toolkit function
# Windows may create an hidden file "Thumbs.db". If the error occurs, run the following funtion
def delete_thumbs_db(directory):
    # Parcours de tous les fichiers et sous-dossiers du r√©pertoire
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file == "Thumbs.db":
                file_path = os.path.join(root, file)
                try:
                    os.remove(file_path)
                    print(f"Supprim√© : {file_path}")
                except Exception as e:
                    print(f"Erreur lors de la suppression de {file_path} : {e}")

# Remplacez 'chemin/vers/dossier' par le chemin r√©el de votre r√©pertoire
delete_thumbs_db("data/carvana/train")
delete_thumbs_db("data/carvana/train_masks")